In [1]:
from __future__ import print_function
"""
Tutorial: A reference implementation of non-hermitian cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# Non-Hermitian Cavity Quantum Electrodynamics Configuration Interaction Singles (NH-CQED-CIS) 

## I. Theoretical Overview

This tutorial builds from the Psi4Numpy tutorial on canonical CIS for molecular systems, that implements the configuration interaction singles method in a spin-adapted basis. The groundwork for working in the spin orbital notation has been laid out in "Introduction to the Spin Orbital Formulation of Post-HF methods" [tutorial](../CQED-Methods/cis_spatial_orbitals.ipynb). It is highly recommended to work through that introduction before starting this tutorial. 

We propose a novel formulation of ab initio CQED that couples an ab initio molecular Hamiltonian to a non-Hermitian photonic Hamiltonian to capture the finite lifetime and energy uncertainty of nanoconfined optical resonances.   As a first realization of this theory, will develop a configuration interaction singles approach for the energy eigenstates of this non-Hermitian polaritonic Hamiltonian, which will allow us to leverage tools of modern quantum chemistry, including analytic evaluation of forces and nonadiabatic couplings, to study polaritonic structure and reactivity.  This will pave the way for future developments of a hierarchy of CI-based approaches that can be systematically improved in terms of their accuracy, and are also interoperable with the powerful machinery of quantum chemistry codes.  In particular, we envision that Complete Active Space CI theory, which has been a particularly promising approach for simulating photochemistry, can be merged with NH-CQED to provide a powerful tool for simulating polaritonic chemistry.
In the following presentation of NH-CQED-CIS theory, we will consider only a single photonic degree of freedom for notational simplicity, but generalizations to additional photonic modes is feasible. 


## NH-CQED-CIS

In CIS, only single excitations from the occupied (indices i,j,k...) to the virtual (indices a,b,c...) orbitals are included. As a result, CIS gives transition energies to an excited state. 

The polaritonic energy eigenfunctions for state $I$ in the
CQED-CIS ansatz can be written as 
\begin{equation}
\Psi_I = c_{0,0} |\Phi_0\rangle |0\rangle + 
c_{0,1} |\Phi_0\rangle |1\rangle +
c^a_{i,0} |\Phi_i^a\rangle |0\rangle +
c^a_{i,1} |\Phi_i^a\rangle |1\rangle 
\end{equation}

Assuming we are using the spin-adapted determinants based on canonical Hartree-Fock orbitals ($\{\mathrm{\psi_p}\}$) with orbital energies $\{\epsilon_p\}$, we can build a CIS Hamiltonian matrix:
$$ \langle s| \langle \Phi_P | \hat{H}|\Phi_Q \rangle | t \rangle  $$

where
\begin{equation}
\Phi_P, \Phi_Q \in {\Phi_0, \Phi_i^a} \; {\rm and} \; |s\rangle, |t\rangle \in {|0\rangle, |1\rangle}
\end{equation}

There are two unique blocks of of the CIS Hamiltonian matrix: 
$$\langle s | \langle \Phi_0 | \hat{H} | \Phi_i^a \rangle | t \rangle$$
and
$$\langle s | \langle \Phi_i^a | \hat{H} | \Phi_j^b \rangle | t \rangle. $$
Each block contains contributions from the molecular electronic degrees of freedom, the photonic degrees
of freedom, and the interaction between the two.

\begin{equation}
\langle s | \langle \Phi_0 | \hat{H} |\Phi_j^b \rangle | t \rangle = 
\langle s | \langle \Phi_0 | \hat{H}_e|\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_0 | \hat{H}_p |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_0 | \hat{H}_{dse} |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_0 | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle.
\end{equation}

\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H} |\Phi_j^b \rangle | t \rangle = 
\langle s | \langle \Phi_i^a | \hat{H}_e|\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_{dse} |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle.
\end{equation}

Specifically, we write the Pauli-Fierz Hamiltonian in the dipole approximation following
Equation (28) of [this work](https://journals.aps.org/prx/pdf/10.1103/PhysRevX.10.041043):
\begin{equation}
    \hat{H} = \hat{H}_e + \hat{H}_p + \hat{H}_{dse} + \hat{H}_{ep} 
    = \hat{H}_e
    + \left( \omega - i \frac{\gamma}{2} \right)\hat{b}^{\dagger} \hat{b} 
    + \frac{1}{2} \left( \lambda \left( \hat{\mu} - \langle \mu \rangle \right) \right)^2
    - \sqrt{\frac{\omega}{2}}\left( \lambda \left( \hat{\mu} - \langle \mu \rangle \right) \right) \left(\hat{b}^{\dagger} + \hat{b}\right) 
\end{equation}
where

$$  \hat{H}_e = \sum_i^{N_e} \hat{T}_e(x_i) + \sum_i^{N_e} \sum_A^{N_N} \hat{V}_{eN}(x_i; X_A) + \sum_i^{N_e} \sum_j^{N_e} \hat{V}_{ee}(x_i, x_j) + V_{N,N} $$
with $\hat{T}_e(x_i)$ denoting the electronic kinetic energy operator for electron $i$, $\hat{V}_{eN}(x_i; X_A)$ the (attractive) coulomb operator for electron $i$ and nucleus $A$, $ \hat{V}_{ee}(x_i, x_j) $ the 
(repulsive) coulomb operator for electrons $i$ and $j$, and $V_{N,N}$ is the total (repulsive) coulomb potential between all of the nuclei.  Within the Born-Oppenheimer approximation, $V_{N,N}$ is a constant, the nuclear 
kinetic energy is neglected, and the electron-nuclear attraction depends parametrically on the fixed nuclear coordinates.

The photonic contribution is captured by the complex energy 

$$ \hat{H}_p = \left(\omega - i \frac{\gamma}{2}\right)\hat{b}^{\dagger} \hat{b} $$ 

and photon-molecule
intereaction contains the bilinear coupling 

$$ \hat{H}_{ep} = - \sqrt{\frac{\omega}{2}}\left( \lambda \left( \hat{\mu} - \langle \mu \rangle \right) \right) \left(\hat{b}^{\dagger} + \hat{b}\right) $$ 

and the quadratic dipole self energy

$$ \hat{H}_{dse} = \frac{1}{2} \left( \lambda \left( \hat{\mu} - \langle \mu \rangle \right) \right)^2. $$
In the above, $\hat{b}^{\dagger}$ and $\hat{b}$ are the bosonic raising/lowering operators for the photonic
degrees of freedom.



$\langle \mu \rangle$ represents the dipole expectation value computed at the Hartree-Fock level.
The dipole operator has the form (in first quantization):
$$ \hat{\mu} = \sum_{i}^{N_e} \mu(x_i) + \sum_{A}^{N_N} \mu_{nuc}(x_A) $$
where $\mu(x_i)$ depends on electronic coordinates and $\mu_{nuc}(x_A)$ depends on nuclear coordinates.

To compute $\langle \mu \rangle$:

\begin{align}
\langle \mu \rangle &= \langle s |\langle \Phi_0 | \sum_{i}^{N_e} \mu(x_i) + \sum_{A}^{N_N} \mu_{nuc}(x_A) | \Phi_0 \rangle |t \rangle \\
&=\langle s | t \rangle \langle \Phi_0 | \sum_{i}^{N_e} \mu(x_i)  | \Phi_0 \rangle + \langle s | t \rangle \sum_{A}^{N_N} \mu_{nuc}(x_A) \\
&= \left(\sum_{i}^{N_{occ}} \langle i | \mu | i \rangle + \sum_{A}^{N_N} \mu_{nuc}(x_A)\right) \delta_{st},
\end{align}
where $\langle i | \mu | i \rangle$ are the molecular dipole integrals and $\sum_{A}^{N_N} \mu_{nuc}(x_A)$
is the nuclear dipole moment defined by the nuclear coordinates and atomic charges in the molecule.  $|s\rangle$ 
and $|t\rangle$ denote photon occupation states.  The nuclear
dipole moment does not depend upon the atomic orbital basis, or the wavefunction ansatz, but is just a
constant defined by the nuclear geometry.  Hence we will refer to it as $\mu_{nuc}$ throughout.  

### Let's examine the quadratic dipole self energy $\hat{H}_{dse}$:

$$\hat{H}_{dse} = \frac{1}{2} \lambda^2 \hat{\mu}^2 - \lambda^2 \langle \mu \rangle \hat{\mu} + \frac{1}{2}\lambda^2 \langle \mu \rangle^2. $$

Term 1:
\begin{align}
\frac{1}{2} \lambda^2 \hat{\mu}^2 &= \frac{1}{2} \lambda^2 \left( \sum_{i}^{N_e} \mu(x_i) + \sum_{A}^{N_N} \mu_{nuc}(x_A) \right) \left( \sum_{j}^{N_e} \mu(x_j) + \sum_{B}^{N_N} \mu_{nuc}(x_B) \right) \\
&= \frac{1}{2} \lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j) + \lambda^2 \mu_{nuc} \sum_i^{N_e} \mu(x_i) + \frac{1}{2}\lambda^2 \mu_{nuc}^2 \\
&=  \frac{1}{2}\lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j) + \frac{1}{2}\lambda^2 \sum_i^{N_e} \mu(x_i) \mu(x_i) + \lambda^2 \mu_{nuc} \sum_i^{N_e} \mu(x_i) + \frac{1}{2}\lambda^2 \mu_{nuc}^2 \\
&= \boxed{\frac{1}{2}\lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j) + \frac{1}{2}\lambda^2 \sum_i^{N_e} Q(x_i) + \lambda^2 \mu_{nuc} \sum_i^{N_e} \mu(x_i) + \frac{1}{2}\lambda^2 \mu_{nuc}^2}
\end{align}
In the above, $\frac{1}{2}\lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j)$ denotes products of dipole operators
for electron $i$ and $j$, and is a two-electron operator, while $\frac{1}{2}\lambda^2 \sum_i^{N_e} \mu(x_i) \mu(x_i)$ 
denotes the product of the dipole operator for electron $i$ and is equivalent to the quadrupole operator for
electron $i$.

Term 2:
$$ - \lambda^2 \langle \mu \rangle \hat{\mu} = \boxed{- \lambda^2 \langle \mu \rangle \sum_i^{N_e} \mu(x_i) - \lambda^2 \langle \mu \rangle \mu_{nuc}} $$

Term 3:
$$ \boxed{\frac{1}{2}\lambda^2 \langle \mu \rangle^2}$$

#### Let's consider the matrix elements that will arise between the HF reference and singly-excited determinants and arbitrary photon number states

**Term 1(a) (2-electron contribution):** $$ \frac{1}{2}\lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j) $$

$$ \frac{1}{2}\lambda^2 \langle s | \langle \Phi_0 | \sum_p^{N_e} \sum_q^{N_e} \mu(x_p) \mu(x_q) | \Phi_i^a \rangle |t\rangle= \boxed{\delta_{st} \frac{1}{2}\lambda^2 \sum_n^{N_{occ}} \mu_{ia} \mu_{nn} - \mu_{in} \mu_{na}} $$

where $\mu_{pq}$ are molecular dipole integrals.

**Term 1(b) (1-electron quadrupole contribution):** $$ \frac{1}{2}\lambda^2 \sum_i^{N_e} Q(x_i)  $$

$$ \frac{1}{2}\lambda^2 \langle s | \langle \Phi_0 | \sum_p^{N_e} Q(x_p) | \Phi_i^a \rangle | t \rangle = \boxed{\delta_{st} \frac{1}{2}\lambda^2  Q_{ia}} $$

where $Q_{pq}$ are molecular quadrupole integrals.

**Term 1(c) (1-electron dipole contribution):** 

$$ \lambda^2 \mu_{nuc} \langle s | \langle \Phi_0 | \sum_i^{N_e} \mu(x_i) |  \Phi_i^a \rangle | t \rangle = \boxed{\delta_{st} \lambda^2 \mu_{nuc} \mu_{ia} } $$

**Term 1(d) (nuclear dipole moment squared):** 

$$ \frac{1}{2} \lambda^2 \mu_{nuc}^2 \langle s | \langle \Phi_0 |  \Phi_i^a \rangle | t \rangle = \boxed{0} $$


**Term 2 (1-electron dipole scaled by dipole moment expectation value)**

$$ - \lambda^2 \langle \mu \rangle \langle s | \langle \Phi_0 | \sum_i^{N_e} \mu(x_i)| \Phi_i^a\rangle | t \rangle - \lambda^2 \langle \mu \rangle \mu_{nuc} \langle s | t \rangle \langle \Phi_0 | \Phi_i^a \rangle = \boxed{- \delta_{st} \lambda^2 \langle \mu \rangle \mu_{ia}}$$


**Term 3 ( dipole moment expectation value squared)**

$$ \lambda^2 \langle \mu \rangle^2 \langle  s | t \rangle \langle \Phi_0 | \Phi_i^a\rangle  = \boxed{0}$$



#### Let's consider the matrix elements that will arise between singly-excited determinants and arbitrary photon number states:

**Term 1(a) (2-electron contribution):** $$ \frac{1}{2}\lambda^2 \sum_i^{N_e} \sum_j^{N_e} \mu(x_i) \mu(x_j) $$

$$ \frac{1}{2}\lambda^2 \langle s | \langle \Phi_i^a | \sum_p^{N_e} \sum_q^{N_e} \mu(x_p) \mu(x_q) | \Phi_j^b \rangle | t \rangle = \boxed{\delta_{st} \frac{1}{2}\lambda^2 \left( \mu_{ai} \mu_{jb} - \mu_{ab} \mu_{ji} \right)} $$

where $\mu_{pq}$ are molecular dipole integrals.

**Term 1(b) (1-electron quadrupole contribution):** $$ \frac{1}{2}\lambda^2 \sum_i^{N_e} Q(x_i)  $$

$$ \frac{1}{2}\lambda^2 \langle s |\langle \Phi_i^a | \sum_p^{N_e} Q(x_p) | \Phi_j^b \rangle | t \rangle = \boxed{\delta_{st} \frac{1}{2}\lambda^2 \left( Q_{ab} \delta_{ij} - Q_{ij}\delta_{ab} \right)} $$

where $Q_{pq}$ are molecular quadrupole integrals.

**Term 1(c) (1-electron dipole contribution):** 

$$ \lambda^2 \mu_{nuc}  \langle s  | \langle \Phi_i^a | \sum_i^{N_e} \mu(x_i) |  \Phi_j^b \rangle | t \rangle = \boxed{\delta_{st} \lambda^2 \mu_{nuc} \left( \mu_{ab}\delta_{ij} - \mu_{ij} \delta_{ab}  \right)} $$

**Term 1(d) (nuclear dipole moment squared):** 

$$ \frac{1}{2} \lambda^2 \mu_{nuc}^2 \langle s | t \rangle \langle \Phi_i^a |  \Phi_j^b \rangle = \boxed{\delta_{st} \frac{1}{2} \lambda^2 \mu_{nuc}^2  \delta_{ij} \delta_{ab}} $$


**Term 2 (1-electron dipole scaled by dipole moment expectation value)**

$$ - \lambda^2 \langle \mu \rangle \langle s |  \langle \Phi_i^a | \sum_i^{N_e} \mu(x_i)| \Phi_j^b\rangle | t \rangle  - \lambda^2 \langle \mu \rangle \mu_{nuc} \langle s | t \rangle \langle \Phi_i^a | \Phi_j^b \rangle = \boxed{-\delta_{st} \lambda^2 \langle \mu \rangle \left(\mu_{ab} \delta_{ij} - \mu_{ij} \delta_{ab} \right) - \delta_{st} \lambda^2 \langle \mu \rangle \mu_{nuc} \delta_{ab} \delta_{ij} } $$


**Term 3 ( dipole moment expectation value squared)**

$$ \lambda^2 \langle \mu \rangle^2 \langle s | t \rangle \langle \Phi_i^a | \Phi_j^b\rangle  = \boxed{\lambda^2 \langle \mu \rangle^2 \delta_{st} \delta_{ab} \delta_{ij}}$$



### Let's consider the bilinear molecule-photon coupling $\hat{H}_{ep}$

$$ \hat{H}_{ep} = - \sqrt{\frac{\omega}{2}}\left( \lambda \left( \hat{\mu} - \langle \mu \rangle \right) \right) \left(\hat{b}^{\dagger} + \hat{b}\right) = \boxed{\sqrt{\frac{\omega}{2}} \lambda \langle \mu \rangle \left( \hat{b}^{\dagger} + \hat{b} \right) - \sqrt{\frac{\omega}{2}}\lambda \hat{\mu} \left( \hat{b}^{\dagger} + \hat{b} \right) }$$

#### Let's consider the matrix elements that will arise between the HF reference and singly-excited determinants and arbitrary photon number states

\begin{align}
\sqrt{\frac{\omega}{2}} \lambda \langle \mu \rangle \langle \Phi_0 | \Phi_i^a \rangle \langle s| \langle  \left( \hat{b}^{\dagger} + \hat{b} \right) | t \rangle - \sqrt{\frac{\omega}{2}}\lambda \langle s | \langle \Phi_0 |\sum_i^{N_e} \mu(x_i) \left( \hat{b}^{\dagger} + \hat{b} \right) | \Phi_i^a \rangle | t \rangle \\
= 0 - \sqrt{\frac{\omega}{2}}\lambda \langle s | \hat{b}^{\dagger} + \hat{b} | t \rangle \langle \Phi_0 | \sum_i^{N_e} \mu(x_i) | \Phi_i^a \rangle \\
= - \sqrt{\frac{\omega}{2}}\lambda \langle s | \hat{b}^{\dagger} | t \rangle  \mu_{ia} - \sqrt{\frac{\omega}{2}}\lambda \langle s | \hat{b}| t \rangle  \mu_{ia} \\
=\boxed{- \sqrt{\frac{\omega}{2}}\lambda \delta_{s,t+1} \mu_{ia} - \sqrt{\frac{\omega}{2}}\lambda \delta_{s+1,t}\mu_{ia}}
\end{align}

#### Let's consider the matrix elements that will arise between singly-excited determinants and arbitrary photon number states

\begin{align}
\sqrt{\frac{\omega}{2}} \lambda \langle \mu \rangle \langle \Phi_i^a | \Phi_j^b \rangle \langle s| \langle  \left( \hat{b}^{\dagger} + \hat{b} \right) | t \rangle - \sqrt{\frac{\omega}{2}}\lambda \langle s | \langle \Phi_i^a |\sum_i^{N_e} \mu(x_i) \left( \hat{b}^{\dagger} + \hat{b} \right) | \Phi_j^b \rangle | t \rangle \\
= \sqrt{\frac{\omega}{2}}\lambda \langle \mu \rangle \delta_{ij}\delta_{ab}\left(\delta_{s,t+1} + \delta_{s+1,t} \right) - \sqrt{\frac{\omega}{2}}\lambda \langle s | \hat{b}^{\dagger} + \hat{b} | t \rangle \langle \Phi_i^a | \sum_i^{N_e} \mu(x_i) | \Phi_j^b \rangle \\
= \boxed{\sqrt{\frac{\omega}{2}}\lambda \langle \mu \rangle \delta_{ij}\delta_{ab}\left(\delta_{s,t+1} + \delta_{s+1,t} \right) - \sqrt{\frac{\omega}{2}}\lambda   \mu_{ab}\delta_{ij} \left( \delta_{s,t+1} + \delta_{s+1,t}\right) +  \sqrt{\frac{\omega}{2}}\lambda   \mu_{ij}\delta_{ab} \left( \delta_{s,t+1} + \delta_{s+1,t}\right)}
\end{align}

Let's now examine the terms that contribute to each block of the CIS Hamiltonian. The 
electronic term is similar to the ordinary CIS matrix elements with the requirement that the photonic
bra and ket states be identical:
\begin{equation}
\boxed{\langle s | \langle \Phi_i^a | \hat{H_e}|\Phi_j^b \rangle | t \rangle = \left((\epsilon_a - \epsilon_i)\delta_{ij} \delta_{ab} + 2(ia|jb) - (ij|ab) \right) \delta_{st}.}
\end{equation}
The elements of $\langle s | \langle \Phi_0 | \hat{H_e}|\Phi_j^b \rangle | t \rangle$ in the canonical Hartree-Fock MO basis.  

The photonic term vanishes unless both the photonic and molecular bra and ket states are identical,
so only one block survives here as well:
\begin{equation}
\boxed{\langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle = \delta_{st} \delta_{ij} \delta_{ab} \left( \hbar \omega + i\frac{\gamma}{2}\right)\sqrt{s}.}
\end{equation}
The bilinear interaction couples particular transitions between photonic and molecular bra and ket states:
\begin{equation}
\boxed{\langle s | \langle \Phi_0 | \hat{H}_{ep}|\Phi_i^a \rangle | t \rangle =
- \sqrt{\frac{\omega}{2}}\lambda \delta_{s,t+1} \mu_{ia} - \sqrt{\frac{\omega}{2}}\lambda \delta_{s+1,t}\mu_{ia}}
\end{equation}

\begin{equation}
\boxed{\langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle =
\sqrt{\frac{\omega}{2}}\lambda \langle \mu \rangle \delta_{ij}\delta_{ab}\left(\delta_{s,t+1} + \delta_{s+1,t} \right) - \sqrt{\frac{\omega}{2}}\lambda   \mu_{ab}\delta_{ij} \left( \delta_{s,t+1} + \delta_{s+1,t}\right) +  \sqrt{\frac{\omega}{2}}\lambda   \mu_{ij}\delta_{ab} \left( \delta_{s,t+1} + \delta_{s+1,t}\right)}
\end{equation}

Finally,
\begin{equation}
\boxed{\langle s | \langle \Phi_0 | \hat{H}_{dse}|\Phi_i^a \rangle | t \rangle = 
\delta_{st} \left( \frac{1}{2}\lambda^2 \sum_n^{N_{occ}} \mu_{ia} \mu_{nn} - \mu_{in} \mu_{na} \right) + 
\delta_{st} \frac{1}{2}\lambda^2  Q_{ia} + 
\delta_{st} \lambda^2 \mu_{nuc} \mu_{ia} - 
\delta_{st} \lambda^2 \langle \mu \rangle \mu_{ia}}
\end{equation}
and
\begin{equation}
\boxed{\langle s | \langle \Phi_i^a | \hat{H}_{dse}|\Phi_j^b \rangle | t \rangle = 
\delta_{st} \frac{1}{2}\lambda^2 \left( \mu_{ai} \mu_{jb} - \mu_{ab} \mu_{ji} \right)
+ \delta_{st} \frac{1}{2}\lambda^2 \left( Q_{ab} \delta_{ij} - Q_{ij}\delta_{ab} \right)
+ \delta_{st} \lambda^2 \mu_{nuc} \left( \mu_{ab}\delta_{ij} - \mu_{ij} \delta_{ab}  \right)
+ \delta_{st} \frac{1}{2} \lambda^2 \mu_{nuc}^2  \delta_{ij} \delta_{ab}
- -\delta_{st} \lambda^2 \langle \mu \rangle \left(\mu_{ab} \delta_{ij} - \mu_{ij} \delta_{ab} \right) - \delta_{st} \lambda^2 \langle \mu \rangle \mu_{nuc} \delta_{ab} \delta_{ij}
+ \lambda^2 \langle \mu \rangle^2 \delta_{st} \delta_{ab} \delta_{ij}}
\end{equation}

\begin{equation}
\tilde{\textbf{H}} \textbf{c}_K = \Delta E_K\textbf{c}_K, \,\Delta E_K = E_K - E_0
\end{equation}
